In [ ]:
# !pip install transformers==2.8.0
!pip install torch==1.4.0

     ━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/753.4 MB 243.3 kB/s eta 0:46:14

In [2]:
# Filtering out the warnings
import warnings
warnings.filterwarnings('ignore')

# Import libraries 

In [3]:
import pandas as pd

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [6]:
from torchtext.models import T5_BASE_GENERATION
transform = T5_BASE_GENERATION.transform()

ImportError: cannot import name 'T5_BASE_GENERATION' from 'torchtext.models' (/home/ahmed/anaconda3/lib/python3.7/site-packages/torchtext/models/__init__.py)

In [5]:
!pip3 install torchtext


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 117.0 kB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [38]:
# initialize the pretrained model
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

# Read Data

In [7]:
df = pd.read_csv('1429_1.csv')
df.columns

Index(['id', 'name', 'asins', 'brand', 'categories', 'keys', 'manufacturer',
       'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen',
       'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username'],
      dtype='object')

In [8]:
df = df[['asins','reviews.rating','reviews.text']]
df.dropna(inplace = True)
df.head()

,asins,reviews.rating,reviews.text
0,B01AHB9CN2,5.0,This product so far has not disappointed. My c...
1,B01AHB9CN2,5.0,great for beginner or experienced person. Boug...
2,B01AHB9CN2,5.0,Inexpensive tablet for him to use and learn on...
3,B01AHB9CN2,4.0,I've had my Fire HD 8 two weeks now and I love...
4,B01AHB9CN2,5.0,I bought this for my grand daughter when she c...


In [9]:
df = df.reset_index(drop=True)

In [10]:
# Choose one product
product1 = df.where(df['asins'] == 'B01AHB9CN2')
product1 = product1.dropna()
product1 = product1.reset_index(drop=True)

In [11]:
# split reviews into pos & neg
pos_rev = ''
neg_rev = ''
for i in range(len(product1)):
    if df['reviews.rating'][i] > 3.0:
        pos_rev = pos_rev + (str(df['reviews.text'][i]))
    elif df['reviews.rating'][i] <= 3.0:
        neg_rev = neg_rev + (str(df['reviews.text'][i]))

In [12]:
# preprocess the pos input text
pos_preprocessed_text = pos_rev.strip().replace('\n','')
pos_t5_input_text = 'summarize: ' + pos_rev

In [13]:
# preprocess the neg input text
neg_preprocessed_text = pos_rev.strip().replace('\n','')
neg_t5_input_text = 'summarize: ' + neg_rev

In [14]:
len(pos_t5_input_text.split())

71589

In [15]:
len(neg_t5_input_text.split())

6581

In [33]:
pos_tokenized_text = tokenizer.encode(pos_t5_input_text, return_tensors='pt', max_length=512).to(device)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [17]:
neg_tokenized_text = tokenizer.encode(neg_t5_input_text, return_tensors='pt', max_length=512).to(device)

# summarize

In [18]:
pos_summary_ids = model.generate(pos_tokenized_text, min_length=30, max_length=120)
pos_summary = tokenizer.decode(pos_summary_ids[0], skip_special_tokens=True)

In [19]:
neg_summary_ids = model.generate(neg_tokenized_text, min_length=30, max_length=120)
neg_summary = tokenizer.decode(neg_summary_ids[0], skip_special_tokens=True)

In [20]:
pos_summary

'amazon fire 8 inch tablet is the perfect size for e-reading on the go. great for e-reading on the go, nice and light weight, and definitely worth the purchase.'

In [21]:
neg_summary

'i bought this tablet for my 4 year old daughter. it is too tied to amazon. It requires too many hoops to get apps to work.'

In [39]:
import pickle

In [40]:
with open("T5-small.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

In [41]:
with open("T5-small-model.pkl", "wb") as f:
    pickle.dump(model, f)